DP算法通常步骤：
1. 刻画一个最优解的结构特征
2. 递归定义最优解的值
3. 计算最优解的值，通常采用自底向上的方法
4. 利用计算出的信息构造一个最优解

## 15.1 钢条切割
**问题描述：**
给定一段长度为 n 英寸的钢条和一个价格表 p<sub>i(i = 1,2,...,n),求切割钢条方案，使得销售收益 r<sub>n 最大。
    
钢条切割问题满足最优子结构性质：问题的最优解又相关子问题的最优解组合而成，而这些子问题可以独立求解。



*自顶向下递归求解（非动态规划）：*

In [7]:
import math
def cut(p,n):
    if n == 0:
        return 0
    q = -math.inf
    for i in range(1,n+1):
        q = max(q,p[i]+cut(p,n-i))
    return q   

以上方法的时间复杂度为 2<sup>n  



动态规划有两种等价的实现方法  
第一种方法称为**带备忘的自顶向下法**。此方法仍按自然的递归形式编写过程，但过程会保存每个子问题的解（数组/散列表）。当需要一个子问题的解时，过程首先检查是否已经保存过此解。如果是，则直接返回保存的值，从而节省了计算时间；否则，按通常方式计算这个子问题。  
第二种方法称为**自底向上法**。这种方法一把需要恰当定义子问题“规模”的概念，使得任何子问题的求解都只依赖于“更小的”子问题的求解。因而我们可以将子问题按规模排序，按由小至大的顺序进行求解。当求解某个子问题时，它所依赖的那些更小的子问题都已求解完毕，结果已经保存。每个子问题只需求解一次，当我们求解它时，它的所有前提子问题都已求解完成。  


带备忘的自顶向下解法：

In [1]:
import math
def memocut(p,n):
    r = [-math.inf]*(n+1)
    return memocutaux(p,n,r)
    

def memocutaux(p,n,r):
    if r[n]>=0:
        return r[n]
    if n == 0:
        q = 0
    else:
        q = -math.inf
        for i in range(1,n+1):
            q = max(q,p[i]+memocutaux(p,n-i,r))
    r[n] = q
    return q


自底向上解法：

In [ ]:
import math
def bottomToUp(p,n):
    r = [-math.inf]*(n+1)
    r[0] = 0
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            q = max(q,p[i]+r[j-i])
        r[j] = q
    return r[n]



计算最大收益值同时保存最优解对应的第一段钢条切割长度：

In [1]:
import math
def eXBottomUp(p,n):
    r = [-math.inf]*(n+1)
    s = [0]*(n+1)
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            if q < p[i]+r[j-i]:
                q = p[i]+r[j-i]
                s[j] = i
        r[j] = q
    return [r[n],s[n]]

  
***练习***   
  
15.1-3 对钢条切割问题进行修改，除了切割下的钢条段具有不同价格p<sub>i外，每次切割还要付出固定成本c  
    

In [4]:
import math
def modiCut(p,n,c):
    r = [-math.inf]*(n+1)
    r[0] = 0
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            if i == j:
                q = max(q,p[i])
            else:
                q = max(q,p[i]+r[j-i]-c)
        r[j] = q
    return r[n]



## 15.4 最长公共子序列  

**最长公共子序列问题**：  
给定两个序列 X=(x<sub>1</sub> ,x<sub>2</sub> ,...,x<sub>m</sub> )和 Y=(y<sub>1</sub> ,y<sub>2</sub> ,...,y<sub>n</sub> )，求 X 和 Y 长度最长的公共子序列  
    
令 X=(x<sub>1</sub> ,x<sub>2</sub> ,...,x<sub>m</sub> )和 Y=(y<sub>1</sub> ,y<sub>2</sub> ,...,y<sub>n</sub> )为两个序列，Z=(z<sub>1 </sub>,z<sub>2</sub> ,...z<sub>k</sub> )为 X 和 Y 的任意 LCS。  
    
1. 如果 x<sub>m</sub> = y<sub>n</sub>,则z<sub>k</sub>=x<sub>m</sub>=y<sub>n</sub> 且 Z<sub>k-1</sub> 是 X<sub>m-1</sub> 和 Y<sub>n-1</sub> 的一个 LCS
2. 如果 x<sub>m</sub>$\neq$y<sub>n</sub>, 那么z<sub>k</sub>$\neq$x<sub>m</sub> 意味着 Z 是 X<sub>m-1</sub> 和 Y 的一个 LCS
3. 如果 x<sub>m</sub>$\neq$y<sub>n</sub>, 那么z<sub>k</sub>$\neq$y<sub>n</sub> 意味着 Z 是 X 和 Y<sub>n-1</sub> 的一个 LCS

In [14]:
def lcsLength(X:str,Y:str):
    m = len(X)
    n = len(Y)
    X=" "+X
    Y=" "+Y
    b = [[0] * (n+1) for num in range(m+1)]
    c = [[0] * (n+1) for num in range(m+1)]
    for i in range(1,m+1):
        c[i][0] = 0
    for j in range(0,n+1):
        c[0][j] = 0
    for i in range(1,m+1):
        for j in range(1,n+1):
            if X[i] == Y[j]:
                c[i][j] = c[i-1][j-1]+1
                b[i][j] = "left-up"
            elif c[i-1][j] >= c[i][j-1]:
                c[i][j] = c[i-1][j]
                b[i][j]="up"
            else:
                c[i][j] = c[i][j-1]
                b[i][j]="left"
    return (b,c)

def printLCS(result:tuple,X,i,j):
    b = result[0]
    if i == 0 or j == 0:
        return 
    if b[i][j] == "left-up":
        printLCS(result,X,i-1,j-1)
        print(X[i-1])
    elif b[i][j] == "up":
        printLCS(result,X,i-1,j)
    else:
        printLCS(result,X,i,j-1)

test1 = "AABBDC"
test2 = "DDBDCA"
result = lcsLength(test1,test2)
printLCS(result,test1,6,6)

B
D
C
